In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm  # Progress Bar 출력

In [8]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        output = self.classifier(x)
        return output

In [9]:
def train(model, trainloaer, loss_fn, optimizer, device):
    model.train()
    correct = 0
    running_size = 0
    running_loss = 0

    prograss_bar = tqdm(trainloaer)

    for i, data in enumerate(prograss_bar):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        _, pred = output.max(dim=1)
        correct += pred.eq(labels).sum().item()
        
        running_loss += loss.item() * inputs.size(0) # FIXME:
        running_size += inputs.size(0)

    loss = running_loss / running_size
    acc = correct / running_size
    return loss, acc

In [10]:
def test(model, testloader, device):
    model.eval()
    correct = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    acc = correct / len(testloader.dataset)
    return acc

In [11]:
def main():


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 데이터셋 불러오기
    transform = transforms.Compose([
        transforms.Resize(size=(227, 227)),
        transforms.ToTensor(), #이미지를 pytorch tensors 타입으로 변형, 0.0~1.0 으로 변환
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # rgb, -1~1로 변환
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False) # TODO: fix

    # 모델 초기화 및 하이퍼파라미터 설정
    model = AlexNet().to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # train, test
    num_epochs = 5

    for epoch in range(num_epochs):
        train_loss, train_acc = train(model, trainloader, criterion, optimizer, device)   
        test_acc = test(model, testloader, device)

        print(f'epoch {epoch+1:02d}, train loss: {train_loss:.5f}, train acc: {train_acc:.5f}, test accuracy: {test_acc:.5f}')

In [12]:
if __name__ == '__main__':
    main()

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 782/782 [07:10<00:00,  1.82it/s]


epoch 01, train loss: 1.70445, train acc: 0.36860, test accuracy: 0.47880


100%|██████████| 782/782 [07:22<00:00,  1.77it/s]


epoch 02, train loss: 1.35124, train acc: 0.51156, test accuracy: 0.55760


...

epoch 03, train loss: 1.20114, train acc: 0.57416, test accuracy: 0.62170


100%|██████████| 782/782 [06:25<00:00,  2.03it/s]


epoch 04, train loss: 1.10034, train acc: 0.61204, test accuracy: 0.63230


100%|██████████| 782/782 [05:46<00:00,  2.26it/s]


epoch 05, train loss: 1.03139, train acc: 0.63652, test accuracy: 0.64910


In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print(torch.backends.cudnn.enabled)
print(torch.__version__)

True
NVIDIA RTX A6000
True
2.0.1
